# Atividade 01 - Criando objetos MNE

Dennis Felipe Urtubia e Pedro Perozin

# Carregamento dos dados, labels e descritor

In [4]:
import numpy as np
import json
import mne

data = np.load('files/data.npy')
labels = np.load('files/labels.npy')

desc_file = open('files/descriptor.json')
descriptor = json.loads(desc_file.read())
desc_file.close()

print('Estruturas => dados', data.shape, 'labels', labels.shape)

Estruturas => dados (125, 257, 1205) labels (125,)


# Criação do Objeto MNE

In [35]:
sampling_frequency = 250
ch_names = 257
ch_types = 'eeg'

# primeiramente devemos criar o objeto info
info = mne.create_info(ch_names=ch_names, sfreq=sampling_frequency, ch_types=ch_types)

# por fim a criação do objeto MNE simples
raw = mne.io.RawArray(data[0], info)

Creating RawArray with float64 data, n_channels=257, n_times=1205
    Range : 0 ... 1204 =      0.000 ...     4.816 secs
Ready.
